In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import plotly.express as px
import dash_table
import signal

# Define a function to stop the script after one minute
def stop_running(signum, frame):
    raise Exception("Script took too long to run")

# Read data
data = pd.read_csv('A688_MARA.csv', sep='\t')
data = data[['Material', 'Changed by', 'Net Weight', 'Volume', 'Material Description', 'DIMS0', 'DIMS1', 'DIMS2', 'Prediction']]

# Define function to add color to outlier rows
def highlight_outliers(s):
    return ['background-color: lightcoral' if v == 'Outlier' else 'background-color: none' for v in s]

# Apply highlight_outliers function to entire DataFrame and display with red rows
data_table = data.style.apply(highlight_outliers, axis=1).data

# Generate list of random numbers between 0 and 0.2
noise = [np.random.uniform(0, 0.2) for _ in range(len(data))]

# Add noise as a new column to data
data['Noise'] = noise

# Create scatter plot with outlier points as light red and inlier points as blue
scatter_plot = px.scatter(data, x='Noise', y=None, color='Prediction',
                          color_discrete_map={'Outlier': 'lightcoral', 'Inlier': 'blue'},
                          hover_data={'Material': True})
scatter_plot.update_layout(title='Inliers and Outliers', xaxis_title='', yaxis_title='')

# Count the number of inliers and outliers
counts = data['Prediction'].value_counts()

# Create the pie chart
pie_chart = px.pie(counts, values=counts.values, names=counts.index, color=counts.index,
                   color_discrete_map={'Outlier': 'lightcoral', 'Inlier': 'blue'})
pie_chart.update_layout(title='Number of Inliers and Outliers')

# Combine all components into a Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Data Analysis Dashboard'),

    html.H2('Data Table'),
    dash_table.DataTable(
        id='table',
        columns=[{'name': i, 'id': i} for i in data_table.columns],
        data=data_table.to_dict('records'),
        style_table={'height': '300px', 'overflowY': 'auto'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[
            {'if': {'column_id': 'Prediction', 'filter_query': '{Prediction} eq "Outlier"'},
             'backgroundColor': 'lightcoral'}
        ]
    ),

    html.H2('Scatter Plot'),
    dcc.Graph(id='scatter', figure=scatter_plot),

    html.H2('Pie Chart'),
    dcc.Graph(id='pie', figure=pie_chart)
])

# Set a timeout for one minute
signal.signal(signal.SIGALRM, stop_running)
signal.alarm(60)

# Run the app and catch the timeout exception
try:
    app.run_server(debug=False, port=8050, host='localhost')
except Exception as e:
    print(e)


Dash is running on http://localhost:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8050
Press CTRL+C to quit
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Apr/2023 23:10:05] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Script took too long to run
